In [ ]:
# !pip uninstall echarts-cities-pypkg
# !pip uninstall echarts-countries-pypkg
# !pip uninstall echarts-china-provinces-pypkg
# !pip uninstall echarts-china-cities-pypkg
# !pip uninstall echarts-china-counties-pypkg
# !pip uninstall echarts-china-misc-pypkg

!pip install echarts-cities-pypkg
!pip install echarts-countries-pypkg
!pip install echarts-china-provinces-pypkg
!pip install echarts-china-cities-pypkg
!pip install echarts-china-counties-pypkg
!pip install echarts-china-misc-pypkg
!pip install jupyter_echarts_pypkg

In [ ]:
import pathlib
import pandas as pd
a = "zz&wz_data.xlsx" # 乌镇&周庄数据原始数据
b = "zz&wz.xlsx" # 乌镇&周庄数据聚合数据
c = "china_coordinates.csv" # 经纬度数据
e = "total.csv"  # 评论数据
f = "provinces_gdp.csv"
g = "zz&wz_spot_coordinate.xlsx"
# 添加经纬度
ll= pd.read_csv(c,header=None,names=['code','name','lon','lat'])
gdp = pd.read_csv(f,encoding='gbk')

In [ ]:
# 数据处理模块
def gen_data(sheet, flag=None):
    pd.options.mode.chained_assignment = None
    # read data
    spot_coordinate = pd.read_excel(g,sheet_name='spot_coordinates')
    spots = pd.read_excel(g,sheet_name=sheet).replace("——", "")
    # gen spot coordinate mapping
    key = spot_coordinate.spot.tolist()
    lon = spot_coordinate.lon.tolist()
    lat = spot_coordinate.lat.tolist()
    spot_maping = {k:[lon[i],lat[i]] for i,k in enumerate(key)}
    # gen sankey data
    data = []
    cols = spots.columns.tolist()
    flag =(0,len(cols)-1) if not flag else flag
    # for i in range(len(cols)-4):
    for i in range(flag[0],flag[1]):
        print(cols[i],cols[i+1])
        tmp = spots[(spots[cols[i]].isin(spot_maping))& (spots[cols[i+1]].isin(spot_maping))][[cols[i],cols[i+1]]]
        if i%2 == 0:
            top_10 = tmp.groupby(cols[i]).count().sort_values(by=cols[i+1],ascending=False).head(10).index.tolist()
            tmp_top10 = tmp[tmp[cols[i]].isin(top_10)]
            for m, group in tmp_top10.groupby(cols[i]):
                group['freq'] = group.groupby(cols[i+1])[cols[i+1]].transform('count')
                data.extend(group.sort_values(by='freq',ascending=False).drop_duplicates().head(10).values.tolist())
        else:
            top_10 = tmp.groupby(cols[i+1]).count().sort_values(by=cols[i],ascending=False).head(10).index.tolist()
            tmp_top10 = tmp[tmp[cols[i+1]].isin(top_10)]
            for m, group in tmp_top10.groupby(cols[i+1]):
                group['freq'] = group.groupby(cols[i])[cols[i]].transform('count')
                data.extend(group.sort_values(by='freq',ascending=False).drop_duplicates().head(10).values.tolist())
    return (data,spot_maping)

In [ ]:
# 绘制桑基图模块
from pyecharts import Sankey
from pyecharts import configure
configure(output_image='png')

# worker of make sankey
def gen_sankey(sheet,name,flag=None):
    # gen data
    data = gen_data(sheet, flag)[0]
    links = [{"source": i[0],"target": i[1],"value": i[2]} for i in data]
    tmp = set([i[0] for i in data] + [i[1] for i in data])
    nodes = [{'name':i} for i in tmp]
    # gen sankey map
    sankey = Sankey("", width=1200, height=600)
    print(nodes,links)
    sankey.add(
        name,
        nodes=nodes,
        links=links,
        line_opacity=0.2,
        line_curve=0.5,
        line_color="red",
        is_label_show=True,
        label_pos="right"
    )
    sankey.render(path=f"{name}.html")
    return sankey

# gen_sankey('wz',"乌镇流入Top10",flag=(2,3))
# gen_sankey('wz',"乌镇流出Top10",flag=(3,4))
gen_sankey('zz',"周庄流入Top10",flag=(2,3))
# gen_sankey('zz',"周庄流出Top10",flag=(3,4))


In [ ]:
# 绘制 流量迁徙图模块
from pyecharts import GeoLines, Style

# worker of make geolines
def gen_geolines(sheet,name,flag=None):
    # gen data
    data,spot_maping = gen_data(sheet, flag)
    # gen geolines map
    style = Style(
        title_top="#fff",
        title_pos = "center",
        width=1200,
        height=600,
        background_color="#404a59"
    )
    style_geo = style.add(
        is_label_show=True,
        line_curve=0.2,
        line_opacity=0.6,
        legend_text_color="#eee",
        legend_pos="right",
        geo_effect_symbol="plane",  #  'circle', 'rect', 'roundRect', 'triangle', 'diamond', 'pin', 'arrow', 'plane' 
        geo_effect_symbolsize=15,
        label_color=['#a6c84c', '#ffa022', '#46bee9'],
        label_pos="right",
        label_formatter="{b}",
        label_text_color="#eee",
    )
    geolines = GeoLines(name, **style.init_style)
    geolines.add("", data, geo_cities_coords=spot_maping, tooltip_formatter="{b}:{c}",**style_geo)
    geolines.render(path=f"{name}.html")
    return geolines

gen_geolines('wz',"乌镇景点迁徙图",flag=(3,4))

In [ ]:
from pyecharts import GeoLines, Style

data_guangzhou = [
    ["广州", "上海", 10],
    ["广州", "北京", 20],
    ["广州", "南京", 30],
    ["广州", "重庆", 40],
    ["广州", "兰州", 50],
    ["广州", "杭州", 60],
]
lines = GeoLines("GeoLines 示例", **style.init_style)
lines.add(
    "从广州出发", data_guangzhou, tooltip_formatter="{a} : {c}", **style_geo
)
lines.render()
lines

In [ ]:
# 处理city表
zz = pd.read_excel(a,sheet_name='zz_city')
wz = pd.read_excel(a,sheet_name='wz_city')
zz.rename(columns={'count': 'zz_count', 'pct': 'zz_pct'}, inplace = True)
wz.rename(columns={'count': 'wz_count', 'pct': 'wz_pct'}, inplace = True)
city = ll[['name','lon','lat']]
city.rename(columns={'name': 'city'}, inplace = True)

df = pd.merge(zz,wz,how='outer').fillna(value={'zz_count':0,'zz_pct':0.0})
df.city.replace({'哈密地区':'哈密市','海西蒙古族藏族自治州直辖':'海西蒙古族藏族自治州','吐鲁番地区':'吐鲁番市',
                 '日喀则地区':'日喀则市','林芝地区':'林芝市','山南地区':'山南市','昌都地区':'昌都市'},inplace=True)
df.zz_count=df.zz_count.astype(int)
df=pd.merge(df,city,how='left')
df.info()

with pd.ExcelWriter(b, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='city',index=False)

In [ ]:
# 处理省份表
zz_prov = pd.read_excel(a,sheet_name='zz_prov')
wz_prov = pd.read_excel(a,sheet_name='wz_prov')
school = pd.read_excel(a,sheet_name='school_prov')
population = pd.read_excel(a,sheet_name='population_prov')
nets = pd.read_excel(a,sheet_name='nets_pct_prov')
cars = pd.read_excel(a,sheet_name='cars_prov')

prov = ll[['name','lon','lat']]
prov.rename(columns={'name': 'prov'}, inplace = True)

zz_prov['prov'].replace('新疆维吾尔自治区', '新疆维吾尔族自治区', inplace=True) 
wz_prov['prov'].replace('新疆维吾尔自治区', '新疆维吾尔族自治区', inplace=True) 
population['prov'].replace({'新疆维吾尔自治区':'新疆维吾尔族自治区','香港':'香港特别行政区','澳门':'澳门特别行政区'}, inplace=True)
population['population']=population['population'].map(lambda x:x.replace('万',''))
nets['prov'].replace('新疆维吾尔自治区', '新疆维吾尔族自治区', inplace=True) 
cars['prov'].replace('新疆维吾尔自治区', '新疆维吾尔族自治区', inplace=True) 
prov['prov'].replace('新疆维吾尔自治区', '新疆维吾尔族自治区', inplace=True) 

zz_prov.rename(columns={'count': 'zz_people_cnt', 'pct': 'zz_people_pct'}, inplace = True)
wz_prov.rename(columns={'count': 'wz_people_cnt', 'pct': 'wz_people_pct'}, inplace = True)
school.rename(columns={'benke': 'bk_school_cnt', 'zhuanke': 'zk_school_cnt','total':'school_cnt'}, inplace = True)
nets.rename(columns={'pct': 'nets_pct'}, inplace = True)

df = pd.merge(zz_prov,wz_prov,how='outer')
df = pd.merge(df,school,how='outer')
df = pd.merge(df,population,how='outer')
df = pd.merge(df,nets,how='outer')
df = pd.merge(df,cars,how='outer').fillna(0)
df = pd.merge(df,prov,how='left')
df.info()
df
with pd.ExcelWriter(b, mode='a', engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='prov',index=False)

In [ ]:
# 处理词云
df = pd.read_csv(e).fillna('')
df.info()
df.replace('用户未点评，系统默认好评。', '',inplace=True)
df['content']=df.content.map(lambda x:x.replace('&#x20', '').replace('取票','').replace('方便','').replace('&#x0A;',''))
# df[df.content== '&#x20']
# df[df.star.isnull()]
# print(df[df.spot=='乌镇'].count())
# df[df.spot=='周庄'].count()
wz_str = ''.join(df[df.spot=='乌镇'].content.tolist())
zz_str = ''.join(df[df.spot=='周庄'].content.tolist())
# wz_str
df[df.spot=='乌镇'].star.mean()
# df[df.spot=='周庄'].star.mean()

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import folium
import webbrowser
from folium.plugins import HeatMap
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import jieba
# sns.boxplot(data=df[df.spot=='乌镇'])
# plt.show()

df[df.spot=='周庄'].star.mean()
# df[df.spot=='乌镇'].star.mean()

star_z = df[df.spot=='周庄'].groupby('star').count()
star_w = df[df.spot=='乌镇'].groupby('star').count()

In [ ]:
df1 = pd.DataFrame({'score': star_z.index, 'count': star_z.spot, 'name':'zhouzhuang'})
df2 = pd.DataFrame({'score': star_w.index, 'count': star_w.spot, 'name':'wuzhen'})
df3 = df1.append(df2)
sns.lineplot(x='score', y='count', data=df3, hue='name')
plt.show()

In [ ]:
# !pip install stop_words
from stop_words import get_stop_words  ## no chinese stop words

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba,io
stop_words = [line.strip().decode('utf-8') for line in io.open('stop_words.txt').readlines()]+['我们','特别','不错','非常','真的','比较']
# token_pattern = ''
# tfidf_vec = TfidfVectorizer(stop_words=stop_words, token_pattern=token_pattern)
tfidf_vec = TfidfVectorizer(stop_words=stop_words)



# 不重复的词
# tfidf_vec.get_feature_names()
# 每个单词的ID

# 每个单词的tfidf值
tfidf_vec = TfidfVectorizer(stop_words=stop_words)
zz_comment = df[df.spot=='周庄'].content.tolist()[:1000]
zz_tmp = [" ".join(jieba.cut(i)) for i in zz_comment]
zz_tfidf_matrix = tfidf_vec.fit_transform(zz_tmp)
zz_tfidfs = zz_tfidf_matrix.toarray()

ids = tfidf_vec.vocabulary_
words = {v: k for k, v in ids.items()}

zz_keys_words = " ".join([words.get(list(i).index(max(i))) for i in zz_tfidfs])
zz_wordcloud = WordCloud(background_color="white",width=1000, height=860, margin=2,
                      font_path="simsun.ttf").generate(zz_keys_words)
plt.imshow(zz_wordcloud)
plt.axis("off")
plt.show()
# zz_wordcloud.to_file('zz_wordcloud.png')

tfidf_vec = TfidfVectorizer(stop_words=stop_words)
wz_comment = df[df.spot=='乌镇'].content.tolist()[:1000]
wz_tmp = [" ".join(jieba.cut(i)) for i in wz_comment]
wz_tfidf_matrix = tfidf_vec.fit_transform(wz_tmp)

ids = tfidf_vec.vocabulary_
words = {v: k for k, v in ids.items()}

wz_tfidfs = wz_tfidf_matrix.toarray()
wz_keys_words = " ".join([words.get(list(i).index(max(i))) for i in wz_tfidfs])
wz_wordcloud = WordCloud(background_color="white",width=1000, height=860, margin=2,
                      font_path="simsun.ttf").generate(wz_keys_words)
plt.imshow(wz_wordcloud)
plt.axis("off")
plt.show()
wz_wordcloud.to_file('wz_wordcloud.png')

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import jieba

# content_str = open('wc.csv').read()

wz_str_tmp = jieba.cut(wz_str)
wz_jieba = " ".join(wz_str_tmp)
zz_str_tmp = jieba.cut(zz_str)
zz_jieba = " ".join(zz_str_tmp)

wz_mask = np.array(Image.open("乌镇.jpg"))
zz_mask = np.array(Image.open("周庄.jpg"))


# wz_wordcloud = WordCloud(background_color="white",width=1000, height=860, margin=2,
#                       font_path="simsun.ttf",mask=wz_mask).generate(wz_str)
# zz_wordcloud = WordCloud(background_color="white",width=1000, height=860, margin=2,
#                       font_path="simsun.ttf",mask=zz_mask).generate(zz_str)
wz_wordcloud = WordCloud(background_color="white",width=1000, height=860, margin=2,
                      font_path="simsun.ttf").generate(wz_str)
zz_wordcloud = WordCloud(background_color="white",width=1000, height=860, margin=2,
                      font_path="simsun.ttf").generate(zz_str)
# width,height,margin可以设置图片属性

# generate 可以对全部文本进行自动分词,但是他对中文支持不好
# 可以通过font_path参数来设置字体集
#background_color参数为设置背景颜色,默认颜色为黑色


plt.imshow(wz_wordcloud)
plt.axis("off")
plt.show()
plt.imshow(zz_wordcloud)
plt.axis("off")
plt.show()

# 保存图片,但是在第三模块的例子中 图片大小将会按照 mask 保存
wz_wordcloud.to_file('wz_wordcloud.png')
zz_wordcloud.to_file('zz_wordcloud.png')


In [ ]:
# 绘制热力图

import numpy as np
import pandas as pd
import seaborn as sns
import folium
import webbrowser
from folium.plugins import HeatMap

city = pd.read_excel(b,sheet_name='city')
prov = pd.read_excel(b,sheet_name='prov')
prov['population']=prov.population.astype(float)

# lat = np.array(city["lat"])
# lon = np.array(city["lon"])
# zz = np.array(city["zz_count"],dtype=float)
# wz = np.array(city["wz_count"],dtype=float)
# data1 = [[lat[i],lon[i],pop[i]] for i in range(num)]    #将数据制作成[lats,lons,weights]的形式

data_zz = city[["lat","lon","zz_count"]].values.tolist()
map_osm = folium.Map(location=[35,110],zoom_start=5)    #绘制Map，开始缩放程度是5倍
HeatMap(data_zz).add_to(map_osm)  # 将热力图添加到前面建立的map里
file_path = "zz_city.html"
map_osm.save(file_path)     # 保存为html文件

data_wz = city[["lat","lon","wz_count"]].values.tolist()
map_osm = folium.Map(location=[35,110],zoom_start=5)    #绘制Map，开始缩放程度是5倍
HeatMap(data_wz).add_to(map_osm)  # 将热力图添加到前面建立的map里
file_path = "wz_city.html"
map_osm.save(file_path)     # 保存为html文件

data_zz_prov = prov[["lat","lon","zz_people_cnt"]].values.tolist()
map_osm = folium.Map(location=[35,110],zoom_start=5)    #绘制Map，开始缩放程度是5倍
HeatMap(data_zz_prov).add_to(map_osm)  # 将热力图添加到前面建立的map里
file_path = "zz_prov.html"
map_osm.save(file_path) 

data_wz_prov = prov[["lat","lon","wz_people_cnt"]].values.tolist()
map_osm = folium.Map(location=[35,110],zoom_start=5)    #绘制Map，开始缩放程度是5倍
HeatMap(data_wz_prov).add_to(map_osm)  # 将热力图添加到前面建立的map里
file_path = "wz_prov.html"
map_osm.save(file_path) 

# webbrowser.open(file_path)  # 默认浏览器打开

In [ ]:
prov.info()
prov


In [ ]:
prov.sort_values(by=['population'],ascending=False,inplace=True)
prov

In [ ]:

gdp.rename(columns={'地区':'prov','2016年':'gdp'},inplace=True)
gdp['prov'].replace('新疆维吾尔自治区', '新疆维吾尔族自治区', inplace=True) 
prov_gdp = pd.merge(prov,gdp[['prov','gdp']], how='left').fillna(0)
prov_gdp

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# 使用Seaborn画折线图
prov_ = prov[~prov['prov'].isin(['浙江省','上海市','江苏省'])]
name =["population","bk_school_cnt","zk_school_cnt","school_cnt","nets_pct","cars"]
for i in name:
    df1 = pd.DataFrame({i: prov_[i], 'people_cnt': prov_.zz_people_cnt, 'name':'zhouzhuang'})
    df2 = pd.DataFrame({i: prov_[i], 'people_cnt': prov_.wz_people_cnt, 'name':'wuzhen'})
    df3 = df1.append(df2)
    sns.lineplot(x=i, y="people_cnt", data=df3, hue='name')
    plt.show()

In [ ]:
prov[~prov['prov'].isin(['浙江省','上海市','江苏省'])]

In [ ]:
# name =["population","bk_school_cnt","zk_school_cnt","school_cnt","nets_pct","cars"]
name =["zz_people_cnt","wz_people_cnt","population","bk_school_cnt","zk_school_cnt","school_cnt","nets_pct","cars",'gdp']
sns.pairplot(prov_gdp[~prov_gdp['prov'].isin(['浙江省','上海市','江苏省','香港特别行政区','澳门特别行政区','台湾省'])][name])
plt.show()

In [ ]:
from sklearn import preprocessing
import numpy as np
min_max_scaler = preprocessing.MinMaxScaler()
# people = min_max_scaler.fit_transform([prov.zz_people_cnt, prov.wz_people_cnt])
# wz_peoples = min_max_scaler.fit_transform(prov[['zz_people_cnt','wz_people_cnt']]).T
# print(wz_peoples)
with sns.axes_style("white"):
#     sns.jointplot(wz_peoples[0], wz_peoples[1], kind="scatter", color="b")
    sns.jointplot(prov_.zz_people_cnt, prov_.wz_people_cnt, kind="reg", color="b")    
    

In [ ]:
sns.swarmplot(x='zz_people_cnt', y='wz_people_cnt', data=prov)

In [ ]:

# prov.info()
# prov
# Seaborn可视化

sns.set(context="notebook")
# 设置风格，seaborn有5种基本风格，context表示环境

x = prov['wz_people_cnt'].astype(int).tolist()
sns.distplot(x, bins = 15, kde=True)
plt.show() 

In [ ]:
 prov.sort_values(by=['zz_people_cnt'], ascending=[True],inplace=True)

In [ ]:
sns.swarmplot(x="total_bill", y="day", hue="time", data=tips)
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
stop_words = ['and', 'is', 'one', 'the', 'this']
# token_pattern = ''
# tfidf_vec = TfidfVectorizer(stop_words=stop_words, token_pattern=token_pattern)
tfidf_vec = TfidfVectorizer(stop_words=stop_words)
documents = [
    'this is the bayes document',
    'this is the second second document',
    'and the third one',
    'is this the document'
]
tfidf_matrix = tfidf_vec.fit_transform(documents)
# 不重复的词
tfidf_vec.get_feature_names()
# 每个单词的ID
tfidf_vec.vocabulary_
# 每个单词的tfidf值
# tfidf_matrix.toarray()

In [ ]:
# !pip install pyecharts pyecharts_snapshot
from pyecharts import GeoLines, Style

style = Style(
    title_top="#fff",
    title_pos = "center",
    width=1200,
    height=600,
    background_color="#404a59"
)

data_guangzhou = [
    ["北京", "上海"],
    ["北京", "北京"],
    ["广州", "南京"],
    ["广州", "重庆"],
    ["北京", "兰州"],
    ["广州", "杭州"]
]

style_geo = style.add(
    is_label_show=True,
    line_curve=0.2,
    line_opacity=0.6,
    legend_text_color="#eee",
    legend_pos="right",
    geo_effect_symbol="plane",
    geo_effect_symbolsize=15,
    label_color=['#a6c84c', '#ffa022', '#46bee9'],
    label_pos="right",
    label_formatter="{b}",
    label_text_color="#eee",
)
geolines = GeoLines("GeoLines 示例", **style.init_style)
geolines.add("从广州出发", data_guangzhou, **style_geo)
geolines.render()
geolines

In [ ]:
from pyecharts import GeoLines, Style
 
#这里先经度后纬度，定义各个点坐标
geo_cities_coords={'三林': [121.5123244, 31.143310800000002],
                   '三林东': [121.5232337, 31.14652508],
                   '三门路': [121.50799520000001, 31.31309147],
                   '上南路': [121.5064128, 31.14911246],
                   '上大路': [121.40917900000001, 31.31352358],
                   '上海体育场': [121.44371310000001, 31.18552163],
                   '上海体育馆': [121.4370549, 31.18272248],
                   '上海儿童医学中心': [121.5239264, 31.20405048]}  
 
 
style = Style(
    title_top="#fff",
    title_pos = "center",
    width=1200,
    height=600,
    background_color="#404a59"
)
 
dataline=[["三林","上海儿童医学中心"],
         ["三林东","上海体育馆"],
         ["三门路","上海体育场"],
         ["上南路","上大路"]]
 
geolines = GeoLines("GeoLines 示例", **style.init_style)
geolines.add("", dataline, maptype = '上海',geo_cities_coords=geo_cities_coords)
geolines.render()
geolines

In [ ]:
from pyecharts import Geo

data = [
    ("海门", 9),("鄂尔多斯", 12),("招远", 12),("舟山", 12),("齐齐哈尔", 14),("盐城", 15),
    ("赤峰", 16),("青岛", 18),("乳山", 18),("金昌", 19),("泉州", 21),("莱西", 21),
    ("日照", 21),("胶南", 22),("南通", 23),("拉萨", 24),("云浮", 24),("梅州", 25),
    ("文登", 25),("上海", 25),("攀枝花", 25),("威海", 25),("承德", 25),("厦门", 26),
    ("汕尾", 26),("潮州", 26),("丹东", 27),("太仓", 27),("曲靖", 27),("烟台", 28),
    ("福州", 29),("瓦房店", 30),("即墨", 30),("抚顺", 31),("玉溪", 31),("张家口", 31),
    ("阳泉", 31),("莱州", 32),("湖州", 32),("汕头", 32),("昆山", 33),("宁波", 33),
    ("湛江", 33),("揭阳", 34),("荣成", 34),("连云港", 35),("葫芦岛", 35),("常熟", 36),
    ("东莞", 36),("河源", 36),("淮安", 36),("泰州", 36),("南宁", 37),("营口", 37),
    ("惠州", 37),("江阴", 37),("蓬莱", 37),("韶关", 38),("嘉峪关", 38),("广州", 38),
    ("延安", 38),("太原", 39),("清远", 39),("中山", 39),("昆明", 39),("寿光", 40),
    ("盘锦", 40),("长治", 41),("深圳", 41),("珠海", 42),("宿迁", 43),("咸阳", 43),
    ("铜川", 44),("平度", 44),("佛山", 44),("海口", 44),("江门", 45),("章丘", 45),
    ("肇庆", 46),("大连", 47),("临汾", 47),("吴江", 47),("石嘴山", 49),("沈阳", 50),
    ("苏州", 50),("茂名", 50),("嘉兴", 51),("长春", 51),("胶州", 52),("银川", 52),
    ("张家港", 52),("三门峡", 53),("锦州", 54),("南昌", 54),("柳州", 54),("三亚", 54),
    ("自贡", 56),("吉林", 56),("阳江", 57),("泸州", 57),("西宁", 57),("宜宾", 58),
    ("呼和浩特", 58),("成都", 58),("大同", 58),("镇江", 59),("桂林", 59),("张家界", 59),
    ("宜兴", 59),("北海", 60),("西安", 61),("金坛", 62),("东营", 62),("牡丹江", 63),
    ("遵义", 63),("绍兴", 63),("扬州", 64),("常州", 64),("潍坊", 65),("重庆", 66),
    ("台州", 67),("南京", 67),("滨州", 70),("贵阳", 71),("无锡", 71),("本溪", 71),
    ("克拉玛依", 72),("渭南", 72),("马鞍山", 72),("宝鸡", 72),("焦作", 75),("句容", 75),
    ("北京", 79),("徐州", 79),("衡水", 80),("包头", 80),("绵阳", 80),("乌鲁木齐", 84),
    ("枣庄", 84),("杭州", 84),("淄博", 85),("鞍山", 86),("溧阳", 86),("库尔勒", 86),
    ("安阳", 90),("开封", 90),("济南", 92),("德阳", 93),("温州", 95),("九江", 96),
    ("邯郸", 98),("临安", 99),("兰州", 99),("沧州", 100),("临沂", 103),("南充", 104),
    ("天津", 105),("富阳", 106),("泰安", 112),("诸暨", 112),("郑州", 113),("哈尔滨", 114),
    ("聊城", 116),("芜湖", 117),("唐山", 119),("平顶山", 119),("邢台", 119),("德州", 120),
    ("济宁", 120),("荆州", 127),("宜昌", 130),("义乌", 132),("丽水", 133),("洛阳", 134),
    ("秦皇岛", 136),("株洲", 143),("石家庄", 147),("莱芜", 148),("常德", 152),("保定", 153),
    ("湘潭", 154),("金华", 157),("岳阳", 169),("长沙", 175),("衢州", 177),("廊坊", 193),
    ("菏泽", 194),("合肥", 229),("武汉", 273),("大庆", 279)]

geo = Geo("地理坐标系示例",
            title_color="#fff",
            title_pos="center",
            width=1200,
            height=600,
            background_color="#404a59",
          )

attr,value = geo.cast(data) # 将内容拆成k,v两个列表

geo.add("",attr,value,
        type="scatter",
        maptype='china', # 地图类型。 从 v0.3.2+ 起，地图已经变为扩展包，支持全国省份，全国城市，全国区县，全球国家等地图
        coordinate_region='中国', # TODO 城市坐标所属国家
        symbol_size=12, # 点的大小
        border_color="#111",
        geo_normal_color="#323c48",
        geo_emphasis_color="#2a333d",
        geo_cities_coords=None, # TODO 支持自定义经纬度，数据类型dict {"城市"：[122,200]}
        is_roam=True, # 缩放，拖拽
        visual_range=[0, 200],
        visual_text_color="#fff",
        is_visualmap=True,
        )
geo

In [ ]:
import os
import json
import codecs

from pyecharts import Sankey

with codecs.open("energy.json", "r", encoding="utf-8") as f:
    j = json.load(f)

sankey = Sankey("桑基图示例", width=1200, height=600)
sankey.add(
    "sankey",
    nodes=j["nodes"],
    links=j["links"],
    line_opacity=0.2,
    line_curve=0.5,
    line_color="source",
    is_label_show=True,
    label_pos="right",
)
sankey.render()
sankey


In [ ]:
5200* 11000/6200


In [5]:
5523457+54385107+64687405+62809247-180626627

6778589

In [ ]:
!pip install fake_useragent

In [ ]:
from fake_useragent import UserAgent
UserAgent.chrome

In [2]:
eval(aa)

NameError: name 'aa' is not defined

In [3]:
288958530*2

577917060

In [6]:
28037 + 12721 + 110054 + 20213 + 3230 + 50 + 44837 + 231810 + 169117 + 274350 + 240045 + 656800 + 151671 + 320222 + 269562 + 194146 + 253087 + 212484 + 285947 + 468003 + 282899 + 167842 + 307913 + 1007046 + 249377 + 220915 + 364578 + 684317 + 44530 + 1053802 + 2242204 + 81302 + 2889187 + 1658591

15200889

In [8]:
1053802+2242204+2889187+1658591+1007046+684317

9535147

In [12]:
import threading
 
class Person(object):
    def __init__(self):
        print("init person")
 
    def speak(self):
        print("speak")
 
if __name__ == "__main__":
    p = Person()
    while True:
        timer = threading.Timer(5, Person.speak, (p,))
        print("start")
        timer.start()
        timer.join()
        print("after join")

init person
start
speak
after join
start
speak
after join
start
speak
after join
start


KeyboardInterrupt: 

speak
